In [1]:
import pandas as pd

In [2]:
fp_zalun = 'Zalun_1985_2017_daily_DMH&DWIR.csv'

In [3]:
df = pd.read_csv(fp_zalun)
df.rename(columns={"Jan": "DoM"}, inplace=True)

In [4]:
months_in_file = ['Jan','Feb','March','April','May','Juni','Juli','Aug','Sep','Oct','Nov','Dec']
months_correct = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

In [5]:
# get columns of years
year_columns = df.columns[1::].tolist()

# get month range slicer
month_idx = [0]
month_idx.extend(df[df.iloc[:,0].isin(months_in_file)].index)
month_idx.append(df.shape[0])

In [6]:
df_collect = pd.DataFrame()
for idx, _ in enumerate(month_idx[:-1]):    
    month = months_correct[idx]
    
    print(month, month_idx[idx], month_idx[idx+1])
    
    # slice a single month
    df_sel = df.iloc[month_idx[idx]:month_idx[idx+1]]
    
    # drop nan rows in month slice
    df_sel.dropna(axis=0, how='all', subset=year_columns, inplace=True)
    
    for year in df_sel.columns[1::]:
        # create dates column
        dates = year+'-'+month+'-'+df_sel.iloc[:,0].astype(str).str.zfill(2)
        # extract the values
        values = df_sel.loc[:, year]
        
        df_year_month = pd.concat((dates, values), axis=1)
        df_year_month.rename(columns={"DoM": "date", year: 'values'}, inplace=True)
        
        df_collect = pd.concat((df_collect, df_year_month), axis=0, ignore_index=True)

Jan 0 31
Feb 31 61
Mar 61 93


c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


Apr 93 124
May 124 156
Jun 156 187
Jul 187 219
Aug 219 251
Sep 251 282
Oct 282 314
Nov 314 345
Dec 345 377


In [7]:
# assing
df_collect = df_collect.assign(date=pd.to_datetime(df_collect['date'], errors='coerce'))
# all 29 february of non-leap years are invalid dates.
df_collect = df_collect[~pd.isnull(df_collect['date'])]

In [8]:
df_collect.shape

(12053, 2)

In [9]:
df_collect.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12053 entries, 0 to 12077
Data columns (total 2 columns):
date      12053 non-null datetime64[ns]
values    11752 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 282.5 KB


In [10]:
df_collect.head()

,date,values
0,1985-01-01,NaN
1,1985-01-02,NaN
2,1985-01-03,NaN
3,1985-01-04,NaN
4,1985-01-05,NaN


In [11]:
df_collect.tail()

,date,values
12073,2017-12-27,360.0
12074,2017-12-28,357.0
12075,2017-12-29,356.0
12076,2017-12-30,355.0
12077,2017-12-31,352.0


In [12]:
start_date = df_collect.iloc[0]['date']
end_date = df_collect.iloc[-1]['date']

In [13]:
# check if no dates are missing
pd.date_range(start=start_date, end=end_date).difference(df_collect['date'])

DatetimeIndex([], dtype='datetime64[ns]', freq=None)

In [14]:
# filter invalid values
df_collect.set_index('date', inplace=True)
df_collect.dropna(inplace=True)

In [15]:
df_collect.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11752 entries, 1986-01-01 to 2017-12-31
Data columns (total 1 columns):
values    11752 non-null float64
dtypes: float64(1)
memory usage: 183.6 KB


In [16]:
df_collect.head()

,values
date,
1986-01-01,310.0
1986-01-02,306.0
1986-01-03,290.0
1986-01-04,289.0
1986-01-05,289.0


In [17]:
df_collect.shape

(11752, 1)